# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
!pip uninstall requests urllib3 -y

In [ ]:
!pip install requests==2.29 urllib3==1.26

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core import Model
from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-capstone-automl")

#print('Workspace name: ' + ws.name, 
#      'Azure region: ' + ws.location, 
#      'Subscription id: ' + ws.subscription_id, 
#      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [3]:
key = "penguins"
description_text = "Penguin Data from Kaggle for Udacity Course Capstone project"

ds = ws.datasets[key] 


df = ds.to_pandas_dataframe()
df.describe()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
count,344,344,344,344,344,344,344
unique,3,3,165,81,56,95,4
top,Adelie,Biscoe,41.1,17,190,3800,MALE
freq,152,168,7,12,22,12,168


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "capstone-compute"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, )
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [ ]:
#from azureml.core import ScriptRunConfig
#from azureml.widgets import RunDetails

#ws = Workspace.from_config()
#experiment = Experiment(workspace=ws, name='day1-experiment-hello')

#config = ScriptRunConfig(source_directory='./', script='hello.py', compute_target='capstone-compute')

#run = experiment.submit(config)
#RunDetails(run).show()

In [5]:
#project_folder = './pipeline-project'

automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=ds,
                             label_column_name="species",   
                             path = './outputs',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             experiment_timeout_minutes=15,
                             max_concurrent_iterations=4,
                             primary_metric='AUC_weighted'
                            )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
remote_run = exp.submit(automl_config, show_output=True)
remote_run.wait_for_completion()

remote_run

Submitting remote run.
No run_configuration provided, running on capstone-compute with default configuration
Running on remote compute: capstone-compute


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone-automl,AutoML_c8f7767a-a6ba-44cf-b7d8-1e4a0516a2cc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_c8f7767a-a6ba-44cf-b7d8-1e4a0516a2cc',
 'target': 'capstone-compute',
 'status': 'Completed',
 'startTimeUtc': '2024-03-22T03:11:06.389894Z',
 'endTimeUtc': '2024-03-22T03:23:19.789378Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'capstone-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-capstone-automl","subscription_id":"a022d83d-6229-4b5e-b039-b680692436b5","resource_group":"aspatton01-rg","workspace_name":"udacity_ml_azure_capstone","region":"eastus2","compute_target":"caps

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

print(fitted_model) #To print

import joblib  

#joblib.dump(fitted_model, "automl_fitted.pkl")
#joblib.dump(best_run, "automl_bestrun.pkl")
# Print the best run
print(best_run)

# Get all metrics of the best run
#best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
#for metric_name in best_run_metrics:
#    metric = best_run_metrics[metric_name]
#    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1, 2]), estimators=[('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1, oob_score=False, random_state=None

In [10]:
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

precision_score_weighted 0.842273527784897
recall_score_micro 0.8192436974789915
average_precision_score_weighted 0.9273110803960813
average_precision_score_micro 0.9273186816134015
average_precision_score_macro 0.9065395959411985
AUC_weighted 0.9570146646621946
norm_macro_recall 0.7205823192974894
recall_score_macro 0.8137215461983264
f1_score_micro 0.8192436974789915
AUC_micro 0.9597639644092931
f1_score_weighted 0.8176583122759592
accuracy 0.8192436974789915
precision_score_macro 0.8075436654848419
balanced_accuracy 0.8137215461983264
weighted_accuracy 0.8157358614669917
recall_score_weighted 0.8192436974789915
log_loss 0.38528529364020503
AUC_macro 0.9581084739009882
matthews_correlation 0.7284852684118489
precision_score_micro 0.8192436974789915
f1_score_macro 0.796297554414569
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_c8f7767a-a6ba-44cf-b7d8-1e4a0516a2cc_36/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_c8f7767a-a6ba-44cf-b7d8-1e4a0516a

In [11]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            print({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            print(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'task': 'classification', 'is_onnx_compatible': False, 'enable_feature_sweeping': True, 'enable_dnn': False, 'force_text_dnn': False, 'feature_sweeping_timeout': 86400, 'featurization_config': None, 'is_cross_validation': True, 'feature_sweeping_config': {}, 'observer': None, 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/capstone-compute/code/Users/aspatton01/nd00333-capstone'}

prefittedsoftvotingclassifier
{'estimators': ['14', '31', '34', '32', '16', '33', '13'], 'weights': [0.3333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333]}

14 - standardscalerwrapper
{'module_name': 'sklearn.preprocessing._data', 'class_name': 'StandardScaler', 'copy': True, 'with_mean': False, 'with_std': False}

14 - randomforestclassifier
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None

In [12]:
import joblib

capstoneModel = best_run.register_model(model_path='outputs/model.pkl', model_name='CapstoneModel_automl',
                        tags={'Model':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

#capstoneModel = Model(ws, name="CapstoneModel_automl")
#model_path = capstoneModel.download(target_dir="./model.pkl")
#loaded_model = joblib.load(model_path)

print(capstoneModel)

Model(workspace=Workspace.create(name='udacity_ml_azure_capstone', subscription_id='a022d83d-6229-4b5e-b039-b680692436b5', resource_group='aspatton01-rg'), name=CapstoneModel_automl, id=CapstoneModel_automl:1, version=1, tags={'Model': 'Auto ML'}, properties={'Accuracy': '0.8192436974789915'})


In [13]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

CapstoneModel_automl version: 1
	 Model : Auto ML
	 Accuracy : 0.8192436974789915


CapstonePenguinsModel version: 1


Hyperdrive-Capstone-Model version: 1
	 azureml.datastoreId : /subscriptions/a022d83d-6229-4b5e-b039-b680692436b5/resourceGroups/aspatton01-rg/providers/Microsoft.MachineLearningServices/workspaces/udacity_ml_azure_capstone/datastores/workspaceartifactstore


CapstonePenguins version: 1


my_model version: 1


CapstoneBest version: 1


hd_best_model version: 1




## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_it.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')


In [18]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='score_it.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "capstone-service", [capstoneModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-03-22 03:31:10+00:00 Creating Container Registry if not exists.
2024-03-22 03:31:11+00:00 Use the existing image.
2024-03-22 03:31:18+00:00 Submitting deployment to compute..
2024-03-22 03:31:24+00:00 Checking the status of deployment capstone-service..
2024-03-22 03:32:06+00:00 Checking the status of inference endpoint capstone-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://4f9fd563-e1f3-4466-a1fe-53d5d552cf16.eastus2.azurecontainer.io/score
http://4f9fd563-e1f3-4466-a1fe-53d5d552cf16.eastus2.azurecontainer.io/swagger.json



TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import json

#Import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('species')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"island": "Dream", "culmen_length_mm": "52.8", "culmen_depth_mm": "20", "flipper_length_mm": "205", "body_mass_g": "4550", "sex": "MALE"}, {"island": "Dream", "culmen_length_mm": "36.4", "culmen_depth_mm": "17", "flipper_length_mm": "195", "body_mass_g": "3325", "sex": "FEMALE"}, {"island": "Dream", "culmen_length_mm": "45.9", "culmen_depth_mm": "17.1", "flipper_length_mm": "190", "body_mass_g": "3575", "sex": "FEMALE"}, {"island": "Biscoe", "culmen_length_mm": "46.2", "culmen_depth_mm": "14.1", "flipper_length_mm": "217", "body_mass_g": "4375", "sex": "FEMALE"}, {"island": "Biscoe", "culmen_length_mm": "41.3", "culmen_depth_mm": "21.1", "flipper_length_mm": "195", "body_mass_g": "4400", "sex": "MALE"}]}


In [21]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [22]:
# Print results from the inference
print(response.text)

"{\"result\": [\"Chinstrap\", \"Adelie\", \"Chinstrap\", \"Gentoo\", \"Adelie\"]}"


In [23]:
# Print original labels
print(label_df)

181    Chinstrap
34        Adelie
166    Chinstrap
334       Gentoo
61        Adelie
Name: species, dtype: object


TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
print(service.get_logs())

2024-03-22T03:31:49,796851228+00:00 - rsyslog/run 
2024-03-22T03:31:49,803643963+00:00 - gunicorn/run 
2024-03-22T03:31:49,806844770+00:00 | gunicorn/run | 
2024-03-22T03:31:49,808664437+00:00 | gunicorn/run | ###############################################
2024-03-22T03:31:49,810458126+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-03-22T03:31:49,816630017+00:00 | gunicorn/run | ###############################################
2024-03-22T03:31:49,818309884+00:00 | gunicorn/run | 
2024-03-22T03:31:49,819481211+00:00 - nginx/run 
2024-03-22T03:31:49,824416775+00:00 | gunicorn/run | 
2024-03-22T03:31:49,829748140+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-03-22T03:31:49,830588798+00:00 | gunicorn/run | 
2024-03-22T03:31:49,831506612+00:00 | gunicorn/run | 
2024-03-22T03:31:49,832337413+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [25]:
service.delete()

Running
2024-03-22 03:36:50+00:00 Check and wait for operation (c3f18d8d-25f7-47b9-a047-16355eb845b2) to finish.
2024-03-22 03:36:52+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
